# PowerShell

Powershell is a mix of a scripting language and shell that is used in the Windows OS(s). It consists of a number of commandlets (cmdlets) that are written in .NET. Cmdlets are generally in the structure verb-subject (or verb-noun). One of my favourite, Test-NetConnection (tnc) has the verb "test" with the subject "NetConnection". Cmdlets can also be piped | to each other.<br>

The 2 most common verbs are Get and Set, however this is by no means exhaustive. Theres even a list of what is allowed [here](https://docs.microsoft.com/en-us/powershell/scripting/developer/cmdlet/approved-verbs-for-windows-powershell-commands?view=powershell-7) <br>

You may notice above I had Test-NetConnection as tnc. This isn't (just) me being lazy, its known as an alias in powershell and is a perfectly lazy / legitimate way to write a cmdlet. I'd probably avoid this when starting out though, an alias is not as readable as the full cmdlet name and can introduce distractions.

<hr>

## Basic Examples

### Get-Help
If you're stuck, the most obvious point to start is Get-Help. This could be likened to the man command in linux. For my example command above, the syntax is:
> Get-Help tnc

### Get-Command
Up next, the Get-Command lists all the commands on the machine, Including Alias, cmdlets, functions etc (theres probably a lot).
> Get-Command

You can provide arguements to the command however, to filter by verb or command. For example, to get all "Test" verbs:
> Get-Command Test-*

Alternatively, all "NetConnection" commands (theres only one of these though)
> Get-Command *-NetConnection

If you know the alias but want the full command, you can always just provide the alias too
> Get-Command tnc

### Get-ChildItem
The name isnt the clearest for this one, but simply it gets the children of the current object. If you run this without any arguements, it prints the current directory contents. You can use this as a search however (the * will get all files, replace with the name you're after. You can include wildcards too (and I suggest adding this at each end).
> Get-ChildItem -Path "C:\" -File "*" -Recurse

### Get-Content
So you've found a file, but now you want to see it. 
> Get-Content file.txt

### Get-FileHash
Because we check this sort of thing... It defaults to the SHA256 Hash but can be modified. MD5 example below.
> Get-FileHash file.txt -Algorithm MD5

### Counting Results
What if you want to count the number of records? This can be done by encasing the command in brackets and calling the count method
> (Get-Command Test-*).count

<hr>

## Piping Cmdlets
Every result from a cmdlet is an 'object', and in .NET, objects include methods. What we see in cmdlet output is a table, but this is really just a way to display what has been found in a way we can understand.

### Filtering
As the result is an object, we can filter on the property. For example, if we wanted all commands under the NetTCPIP group:
> Get-Command | Where-Object Source -eq NetTCPIP

Or, If we wanted to remove a source (because it spamms everything)
> Get-Command | Where-Object Source -ne AWSPowerShell

The -eq is 'equal', we also have -GT / -LT, -Contains, -cLike (you need this one for *) etc. [Theres more.](https://docs.microsoft.com/en-us/powershell/module/microsoft.powershell.core/where-object?view=powershell-6)

<hr>

## Threat Hunting
So, this is a bit more fun. Most of these are cross posts from the [Sysmon](SysInternals/Sysmon.ipynb) page, and you'll find these same 'hunts' there. Theres also a bit of [EventViewer](BlueTeam/WindowsEventViewer.ipynb) in here.

### Mimikatz
Well.. more specifically the LSASS behaviour that Mimikatz is known for. Bit of background, its dumping the stored credentials from memory. This is done using the LSASS exe but this should only be called from svhost normally. If it's not, you've probably got katz.

> Get-WinEvent -FilterXPath '*/System/EventID=10 and */EventData/Data[@Name="TargetImage"] and */EventData/Data="C:\Windows\system32\lsass.exe"'

This one looks for Event10 (process accessed I think) where the target is lsass.exe. 


### Metasploit
A personal favourite of everyone from script kiddies to professionals, this one highlights the need to not use standard setups; even as an attacker.

> Get-WinEvent -FilterXPath '*/System/EventID=3 and */EventData/Data[@Name="DestinationPort"] and */EventData/Data=4444'

This one looks for Event3 (Network Connection) on port 4444 (outbound). As you should know, this is the default port that Metasploit creates a listener on. 

<hr>